In [10]:
import pandas as pd
import numpy as np
from datetime import datetime

In [11]:
catalog = pd.read_csv('SCEDC_catalog.csv')
catalog.sort_values('time', inplace=True)
catalog.head()

,time,magnitude,depth,longitude,latitude,x_utm,y_utm
0,-1.199085e+09,2.73,6.0,-117.645,33.903,440367.549732,3.751588e+06
1,-1.198995e+09,3.00,6.0,-116.000,32.000,594457.463402,3.540873e+06
2,-1.198895e+09,2.00,6.0,-117.494,33.771,454257.899420,3.736875e+06
3,-1.198771e+09,2.00,6.0,-117.475,34.143,456208.233842,3.778114e+06
4,-1.198660e+09,3.00,6.0,-116.000,32.000,594457.463402,3.540873e+06


In [12]:
target_magnitude = 4

# Train: 1985 - 2004
train_start_time = pd.Timestamp('1985-01-01')
# Validation: 2005 - 2013
validation_start_time = pd.Timestamp('2005-01-01')
# Test: 2014 - 2019
test_start_time = pd.Timestamp('2014-01-01')
test_end_time = pd.Timestamp('2020-01-01')

In [13]:
conditional_catalog = catalog[(catalog['magnitude'] >= target_magnitude)]
conditional_catalog.head()

,time,magnitude,depth,longitude,latitude,x_utm,y_utm
36,-1.195606e+09,4.27,12.0,-116.800,34.344,518394.795719,3.800317e+06
63,-1.192146e+09,4.46,6.0,-115.932,35.795,596506.584311,3.961737e+06
108,-1.189038e+09,4.14,6.0,-116.825,35.078,515953.879512,3.881707e+06
184,-1.181322e+09,4.44,6.0,-118.446,36.074,369793.045384,3.993124e+06
185,-1.180976e+09,4.43,6.0,-116.400,34.871,554837.129912,3.858902e+06


In [14]:
# Training + validation period
train_val_start = train_start_time
train_val_end = test_start_time

In [15]:
# Number of earthquakes in training + validation period
n_train_val = len(conditional_catalog[(conditional_catalog['time'] >= train_val_start.timestamp()) &
                                      (conditional_catalog['time'] < train_val_end.timestamp())])

# Total duration of training + validation period in days
T_train_val = (test_start_time - train_start_time).total_seconds() / (24 * 3600)

# Maximum Likelihood Estimate of the rate
lambda_hat = n_train_val / T_train_val

In [16]:
print(f"Events in training + validation period (n_train_val): {n_train_val}")
print(f"Duration of training + validation period (T_train_val): {T_train_val:.0f} days")
print(f"Estimated Poisson Rate (lambda_hat): {lambda_hat:.4f} earthquakes/day")

Events in training + validation period (n_train_val): 966
Duration of training + validation period (T_train_val): 10592 days
Estimated Poisson Rate (lambda_hat): 0.0912 earthquakes/day


In [17]:
# Number of earthquakes in testing period
n_test = len(conditional_catalog[(conditional_catalog['time'] >= test_start_time.timestamp()) &
                                 (conditional_catalog['time'] < test_end_time.timestamp())])

# Total duration of testing period in days
T_test = (test_end_time - test_start_time).total_seconds() / (24 * 3600)

log_likelihood_test = (n_test * np.log(lambda_hat)) - (lambda_hat * T_test)

In [18]:
print(f"Events in testing period (n_test): {n_test}")
print(f"Duration of testing period (T_test): {T_test:.0f} days")
print(f"Log Likelihood on Test Data: {log_likelihood_test:.4f}")
print(f"Poisson Per-Example Loss: {(-log_likelihood_test / n_test):.4f}") # Negative Log Likelihood

Events in testing period (n_test): 170
Duration of testing period (T_test): 2191 days
Log Likelihood on Test Data: -606.9186
Poisson Per-Example Loss: 3.5701
